# Article Work 
We looked at an article Chakraborty, P. (2021). Energy Management Using Real-Time Non-Intrusive Load Monitoring. [online] Towards Data Science. Available at: https://towardsdatascience.com/energy-management-using-real-time-non-intrusive-load-monitoring-3c9b0b4c8291 [Accessed 23 Apr. 2023]. In which we decided to try and follow the steps and methodolgy used using our Kaggle data as the dataset this is the attempt that was made on this method. 

In [11]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
import numpy as np

In [18]:
import re
# Load the DataFrame
df = pd.read_csv('London_House_data.csv', dtype=dtypes)

# Remove spaces from column names
df.columns = [col.strip() for col in df.columns]

# Convert the DateTime column to pandas datetime objects
df['DateTime'] = pd.to_datetime(df['DateTime'])

# Remove duplicates in the DateTime column
df = df.drop_duplicates(subset='DateTime')

# Set the DateTime column as the index
df = df.set_index('DateTime')

# Resample to 10-minute intervals and interpolate the KWh column
df_resampled = df.resample('10T').interpolate(method='linear', limit_area='inside')

# Forward fill the other columns
df_resampled[['LCLid', 'Acorn', 'House Type', 'Amount of Beds', 'House Value', 'How many People Living there']] = df_resampled[['LCLid', 'Acorn', 'House Type', 'Amount of Beds', 'House Value', 'How many People Living there']].ffill()

# Reset the index
df_resampled = df_resampled.reset_index()
print(df_resampled)
# One-hot encode the 'Acorn' column
acorn_encoder = OneHotEncoder(sparse=False)
acorn_encoded = acorn_encoder.fit_transform(df_resampled['Acorn'].values.reshape(-1, 1))
acorn_labels = acorn_encoder.categories_[0]
# Convert 'Amount of Beds' values to strings
df_resampled['Amount of Beds'] = df_resampled['Amount of Beds'].astype(str).str.strip()
df_resampled['Amount of Beds'] = df_resampled['Amount of Beds'].replace('5+', '5')
print(df_resampled['Amount of Beds'].unique())
df_resampled['Amount of Beds'] = df_resampled['Amount of Beds'].astype(float)
# Convert 'How many People Living there' values to strings
df_resampled['How many People Living there'] = df_resampled['How many People Living there'].astype(str).str.strip()
df_resampled['How many People Living there'] = df_resampled['How many People Living there'].replace('5+', '5')
print(df_resampled['How many People Living there'].unique())
df_resampled['How many People Living there'] = df_resampled['How many People Living there'].astype(float)
def convert_house_value(value):
    def convert_suffix(num_str):
        num_str = num_str.lower()
        if 'k' in num_str:
            return float(num_str.replace('k', '')) * 1_000
        elif 'm' in num_str:
            return float(num_str.replace('m', '')) * 1_000_000
        else:
            return float(num_str)

    if pd.isna(value) or value == 'nan' or value == '0':
        return 0.0

    value = str(value).strip()
    value = re.sub('[^\dkm-]', '', value)  # Remove non-numeric characters except k and m

    if '-' in value:
        low, high = value.split('-')
        low = convert_suffix(low)
        high = convert_suffix(high)
        return (low + high) / 2
    else:
        return convert_suffix(value)

# Convert House Value
df_resampled['House Value'] = df_resampled['House Value'].astype(str)
df_resampled['House Value'] = df_resampled['House Value'].apply(lambda x: convert_house_value(x) if x else 0)
df_resampled['House Value'] = df_resampled['House Value'].astype(float)


df_resampled['House Value'] = df_resampled['House Value'].replace('100k-150k', '150000')
df_resampled['House Value'] = df_resampled['House Value'].replace('150k-200k', '200000')
df_resampled['House Value'] = df_resampled['House Value'].replace('200k-250k', '250000')
df_resampled['House Value'] = df_resampled['House Value'].replace('250k-300k', '300000')
df_resampled['House Value'] = df_resampled['House Value'].replace('300k-350k', '350000')
df_resampled['House Value'] = df_resampled['House Value'].replace('350k-400k', '400000')
df_resampled['House Value'] = df_resampled['House Value'].replace('400k-450k', '450000')
df_resampled['House Value'] = df_resampled['House Value'].replace('450k-500k', '500000')
df_resampled['House Value'] = df_resampled['House Value'].replace('500k-750k', '750000')
df_resampled['House Value'] = df_resampled['House Value'].replace('750k-1m', '1000000')
df_resampled['House Value'] = df_resampled['House Value'].replace('1m+', '1005000')



# Label encode the 'Amount of Beds' column
beds_encoder = LabelEncoder()
beds_encoded = beds_encoder.fit_transform(df_resampled['Amount of Beds'].values)
beds_labels = beds_encoder.classes_

# Label encode the 'Amount of Beds' column
beds_encoder = LabelEncoder()
beds_encoded = beds_encoder.fit_transform(df_resampled['Amount of Beds'].values)
beds_labels = beds_encoder.classes_
def convert_house_value(value):
    def convert_suffix(num_str):
        if 'k' in num_str:
            return float(num_str.replace('k', '')) * 1e3
        elif 'm' in num_str:
            return float(num_str.replace('m', '')) * 1e6
        else:
            return float(num_str)

    value = re.sub('[^\dkm-]', '', value)  # Remove non-numeric characters except k and m
    if '-' in value:
        low, high = value.split('-')
        return (convert_suffix(low) + convert_suffix(high)) / 2
    else:
        return convert_suffix(value)

df_resampled['House Value'] = df_resampled['House Value'].apply(convert_house_value)
# Normalize the 'KWh' and 'House Value' columns
scaler = MinMaxScaler()
df_resampled['KWh_normalized'] = scaler.fit_transform(df_resampled['KWh'].values.reshape(-1, 1))
df_resampled['House Value_normalized'] = scaler.fit_transform(df_resampled['House Value'].values.reshape(-1, 1))

# Add the encoded columns to the DataFrame
df_encoded = pd.DataFrame(acorn_encoded, columns=[f'Acorn_{label}' for label in acorn_labels], index=df_resampled.index)
df_encoded['Amount of Beds'] = beds_encoded
df_encoded['House Value'] = df_resampled['House Value_normalized']
df_encoded['How many People Living there'] = df_resampled['How many People Living there']

# Generate input and target samples using a sliding window
window_size = 6
stride = 1
inputs, targets = generate_samples(df_encoded.values, window_size, stride)

# Print the final DataFrame
print(df_encoded)

C:\Users\ricky\anaconda3\envs\nilmtk\lib\site-packages\IPython\core\interactiveshell.py:3221: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


                        DateTime      LCLid         KWh Acorn  \
0      2011-12-06 15:30:00+00:00  MAC000017  127.000000     C   
1      2011-12-06 15:40:00+00:00  MAC000017  109.666667     C   
2      2011-12-06 15:50:00+00:00  MAC000017   92.333333     C   
3      2011-12-06 16:00:00+00:00  MAC000017   75.000000     C   
4      2011-12-06 16:10:00+00:00  MAC000017   70.666667     C   
...                          ...        ...         ...   ...   
117263 2014-02-27 23:20:00+00:00  MAC000323  816.666667     A   
117264 2014-02-27 23:30:00+00:00  MAC000323  789.000000     A   
117265 2014-02-27 23:40:00+00:00  MAC000323  637.333333     A   
117266 2014-02-27 23:50:00+00:00  MAC000323  485.666667     A   
117267 2014-02-28 00:00:00+00:00  MAC000323  334.000000     A   

               House Type Amount of Beds House Value  \
0        Detached Houses               4    750k-1m    
1        Detached Houses               4    750k-1m    
2        Detached Houses               4    750k-1m

C:\Users\ricky\anaconda3\envs\nilmtk\lib\site-packages\sklearn\preprocessing\_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


['2' '5']


TypeError: expected string or bytes-like object

In [ ]:
#if we use Skilearn instead then Decision Tree could be possible 
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Split the data into training and test sets
split = int(0.75 * len(inputs))
train_inputs, test_inputs = inputs[:split], inputs[split:]
train_targets, test_targets = targets[:split], targets[split:]

# Define the model
model = DecisionTreeRegressor()

# Fit the model to the training data
model.fit(train_inputs, train_targets)

# Make predictions on the test data
predictions = model.predict(test_inputs)

# Calculate the mean squared error
mse = mean_squared_error(test_targets, predictions)
print(f'MSE: {mse:.3f}')

## Failures:
Trying my best to keep up with the article, I had two major issues, the first is that when preprocessing the data There was issues with string elements and my laptop refused to install keras

In [53]:
from keras.models import Model
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error

# Define the model architecture
input_shape = (window_size, inputs.shape[2])
input_layer = Input(shape=input_shape)
conv1_layer = Conv1D(filters=16, kernel_size=3, activation='relu')(input_layer)
pool1_layer = MaxPooling1D(pool_size=2)(conv1_layer)
conv2_layer = Conv1D(filters=32, kernel_size=3, activation='relu')(pool1_layer)
pool2_layer = MaxPooling1D(pool_size=2)(conv2_layer)
flatten_layer = Flatten()(pool2_layer)
dense1_layer = Dense(units=64, activation='relu')(flatten_layer)
dropout_layer = Dropout(rate=0.5)(dense1_layer)
output_layer = Dense(units=1)(dropout_layer)
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(inputs, targets, batch_size=64, epochs=50, validation_split=0.2, callbacks=[early_stop])

# Evaluate the model
predictions = model.predict(inputs)
mse = mean_squared_error(targets, predictions)
print(f'MSE: {mse:.3f}')

ModuleNotFoundError: No module named 'keras'